# City distances

In [2]:
import sys
!{sys.executable} -m pip install openpyxl

import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim
from time import sleep, time
from tqdm import tqdm


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
cities_df = pd.read_csv("./data/cities_with_countries.csv")
cities_df.head()

,city,country_code
0,aalen,DE
1,abanilla,ES
2,ablis,FR
3,abrera,ES
4,agnadello,IT


In [ ]:
geolocator = Nominatim(user_agent="city_geocoder")

city_data = []

start_time = time()

for _, row in tqdm(cities_df.iterrows(), total=len(cities_df), desc="🌍 Geokodowanie miast", unit="city"):
    city = row["city"]
    country_code = row["country_code"]

    try:
        location = geolocator.geocode(city, country_codes=country_code, timeout=10)
        #location = geolocator.geocode(city, timeout=10)
        if location:
            city_data.append({
                "city": city,
                "country_code": country_code,
                "latitude": location.latitude,
                "longitude": location.longitude
            })
        else:
            city_data.append({"city": city, "country_code": country_code, "latitude": None, "longitude": None})
            print(f"⚠️ Nie znaleziono lokalizacji dla miasta: {city} ({country_code})")
    except Exception as e:
        tqdm.write(f"Błąd dla miasta {city}: {e}")
        city_data.append({"city": city, "country_code": country_code, "latitude": None, "longitude": None})
    sleep(0.1)  # bezpieczna pauza, by nie przeciążać API

elapsed = time() - start_time
print(f"\n✅ Geokodowanie zakończone! Łączny czas: {elapsed:.2f} sekund")

df_cities = pd.DataFrame(city_data)
output_path = "./data/cities_latitude_longitude.csv"
df_cities.to_csv(output_path, index=False)
print(f"📁 Wynik zapisano do: {output_path}")


🌍 Geokodowanie miast:  62%|██████▏   | 437/702 [07:16<04:21,  1.01city/s]

⚠️ Nie znaleziono lokalizacji dla miasta: nowe obrzyska (PL)


🌍 Geokodowanie miast: 100%|██████████| 702/702 [11:43<00:00,  1.00s/city]


✅ Geokodowanie zakończone! Łączny czas: 703.81 sekund
📁 Wynik zapisano do: ./data/cities_latitude_longitude.csv


# Detekcje niepoprawnych nazw na podst tych samych współżędnych

In [1]:
cities_df = pd.read_csv("./data/cities_latitude_longitude.csv")
cities_df.head()

,city,country_code,latitude,longitude
0,aalen,DE,48.837561,10.092959
1,abanilla,ES,38.205826,-1.039238
2,ablis,FR,48.517153,1.836876
3,abrera,ES,41.520446,1.902413
4,agnadello,IT,45.444374,9.557056


In [ ]:
# grouped = (
#     cities_df
#     .groupby(["country_code", "latitude", "longitude"])
#     .agg({"city": list, "city": "unique", "city": lambda x: list(set(x))})
#     .reset_index()
# )

duplicates = (
    cities_df
    .groupby(["country_code", "latitude", "longitude"])
    .filter(lambda g: g["city"].nunique() > 1)
    .sort_values(["country_code", "latitude", "longitude"])
)

duplicates

,city,country_code,latitude,longitude


### Poprawa nazw miast

In [18]:
plik = "./data/dane_puste_przebiegi_cleaned.xlsx"

df = pd.read_excel(plik)
df.head()

,full_number,client,tractor,route,load_city,load_postal_code,load_country,unload_city,unload_postal_code,unload_country,pickup_planned,pickup_actual,delivery_planned,delivery_actual,empty_km_map,loaded_km_map,total_km_map
0,26692/2025,129.0,WGM9815L,(DE) Nürnberg - (AT) Wörgl - (DE) Nürnberg,nürnberg,90475,DE,nürnberg,90475,DE,2025-09-08 00:30:00,2025-09-08 00:46:00,2025-09-08 22:00:00,2025-09-08 22:15:00,0.000,553.013,553.013
1,27082/2025,105.0,PZ4S023,(PL) Jarosty - (CZ) Praha 5,jarosty,97-310,PL,praha 5,NaN,CZ,2025-09-08 01:10:00,2025-09-07 23:55:00,2025-09-09 03:15:00,2025-09-09 01:47:27,0.000,540.058,540.058
2,27028/2025,61.0,PZ4R993,(DE) Euskirchen - (FR) amiens,euskirchen,53881,DE,amiens,80013,FR,2025-09-08 05:00:00,2025-09-08 16:32:00,2025-09-08 15:00:00,2025-09-09 11:16:00,0.000,409.626,409.626
3,27026/2025,47.0,WGM8283K,(CZ) Modřice - (CZ) Pelhřimov - (CZ) VRSKMAN,modřice,66442,CZ,vrskman,43111,CZ,2025-09-08 05:30:00,2025-09-08 05:48:28,2025-09-08 12:30:00,2025-09-08 14:04:21,318.425,351.447,669.872
4,27049/2025,105.0,WGM5118H,(PL) Chlastawa - (DE) Berlin,chlastawa,66-210,PL,berlin,13597,DE,2025-09-08 06:00:00,2025-09-09 00:34:00,2025-09-09 06:00:00,2025-09-09 05:17:00,0.789,216.089,216.878


In [19]:
df[["load_city", "unload_city"]] = df[["load_city", "unload_city"]].replace({
    "csomor": "csömör",
    "kolonia": "köln",
    "vieux- thann": "vieux-thann",
    "saint-quentin fallavier": "saint-quentin-fallavier",
    "saint quentin fallavier": "saint-quentin-fallavier",
    "st quentin fallavier": "saint-quentin-fallavier",
    "rakovnik": "rakovník",
    "vaihingen": "vaihingen an der enz",
    "mockmuhl": "möckmühl",
    "moeckmuehl": "möckmühl",
    "buerstadt": "bürstadt",
    "burstadt": "bürstadt",
    "suelzetal": "sülzetal",
    "sulzetal": "sülzetal",
    "henstedt ulzburg": "henstedt-ulzburg",
    "neumunster": "neumünster",
    "moissy cramayel": "moissy-cramayel",
    "trith-saint-leger": "trith-saint-léger",
    "gyal": "gyál",
    "gostyn": "gostyń",

    # inne błędy
    "chateaudun cedex": "châteaudun", # cedex - to oddział poczty (niepotrzebne w nazwie miasta)
    "jübar ot nettgau": "jübar", # Jübar to miejscowość/gmina, a Nettgau to jej część składowa (Ortsteil) — czyli osada / dzielnica / część administracyjna w obrębie Jübar
    "tm nieuwdorp": "nieuwdorp", # literówka
    "seeris": "serris", # literówka
    "striegistal-gebiet berbersdorf": "berbersdorf", # literówka
    
})

In [20]:
df.to_excel(plik, index=False)

print(f"✅ Dane wyczyszczone i zapisane do: {plik}")

✅ Dane wyczyszczone i zapisane do: ./data/dane_puste_przebiegi_cleaned.xlsx
